# Algoritmo para la Generación de Árboles de Desiciones ID3
<br>

<div style="text-align: justify">

En general un árbol de decisiones representa un conjunto de reglas estructuradas jerárquicamente que puede servir para la clasificación y la toma de decisiones.Cada camino de la raíz a una hoja corresponde a una conjunción de evaluación de atributos  y el árbol en si es una disyunción de estas conjunciones.La mayoría de los algoritmos que han sido desarrollados para el aprendizaje mediante árboles de decisión son variantes de un algoritmo núcleo que emplea una búsqueda voraz top-down a través del espacio de los posibles árboles de decisión. Este acercamiento fue ejemplificado por los algoritmos ID3 (Quinlan 1986) y por su sucesor C4.5 (Quinlan 1993)

El conjunto de ejemplos deberá estar conformado por una serie de tuplas de valores, cada uno de ellos denominados atributos, en el que uno de ellos, ( el atributo a clasificar ) es el objetivo, el cual es de tipo binario( positivo o negativo, sí o no, válido o inválido. Tambié se debe tener en cuenta que ID3 solo trabaja con atributos nominales. ID3 realiza su labor mediante la construcción de un árbol de decisión, cuyos elementos son: 
</div> 
<ol>
<li>Nodos: Los cuales contendrán atributos.</li>
<li>Arcos: Los cuales contienen valores posibles del nodo padre. </li>
<li>Hojas: Nodos que clasifican el ejemplo como positivo o negativo (valores del atributo clase)</li>
</ol>

<br>
<div style="text-align: justify">


La heurística del algoritmo se basa en seleccionar el atributo que mayor incida en la clasificación para ello usaremos el estadístico llamado Ganancia de Información.Este nos da una medida de efectividad de un atributo en la clasificación del conjunto de entrenamiento.
La Ganancia de Información a su vez utiliza la entrepía de de información que nos da la medida de la cantidad de ruido o desorden que contiene o libera un sistema. Caracteriza la (im)pureza de una colección arbitraria de datos. Por tanto la Ganancia de Información es la reducción esperada en la entropía del conjunto casusada por conocer un atributo A cualquiera. A continuación se implementa el algoritmo ID3 utilizando la biblioteca Graphviz para la estructura de datos de árbol y la visualización.
</div> 


In [1]:
import pandas as pd
import numpy as np
from scipy import stats
import statsmodels.formula.api as formula
import matplotlib.pyplot as plt
%matplotlib inline
from __future__ import division, print_function
import random
import sklearn
import pylab
import scipy.stats as stats
from sklearn.naive_bayes import GaussianNB
from sklearn.cross_validation import train_test_split
from math import exp, expm1
import graphviz as gv
from graphviz import Digraph
import random
# Cargando el dataset de muestra
emergencias = pd.read_csv("emergencias.csv",sep=',',encoding="utf-8",engine='python')
emergencias.head()

/usr/local/lib/python3.5/dist-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


,Presion arterial,Azucar en sangre,Indice de colesterol,Alergia a antibioticos,Otras alergia,Administrar farmaco F
0,Alta,Alto,Alto,No,No,Si
1,Alta,Alto,Alto,Si,No,Si
2,Baja,Alto,Bajo,No,No,Si
3,Media,Alto,Alto,No,Si,No
4,Media,Bajo,Alto,Si,Si,No


In [4]:

# Devuelve un arreglo de valores y número de ocurrencias del valor
def Hallar_Valores(A,S):
    return pd.Series(S[A].as_matrix()).value_counts()
# Calcula la entropía del dataset
def Entropia(S, A):
    Valores= Hallar_Valores(A,S)
    #print(Valores)
    T=sum(Valores)
    E=0
    # Sumatoria -P*log2(P)
    for i in range (0, Valores.count()):
        p=Valores[i]/T
        E +=-(p*np.log2(p))
    return E

# Calcula la ganacia de información
def Ganancia(S,A,A_clase):
    Entropia_S= Entropia(S, A_clase)
    # Hallando valores del atributo clase y valores del atributo   
    Valores_clase= Hallar_Valores(A_clase,S)
    Valores_atributo= Hallar_Valores(A,S)
    Entropia_A_con_Valor_i=0
    Ganancia=Entropia_S
    C=""
    
    for i in  range(0,Valores_atributo.count()):
        for j in range(0,Valores_clase.count()):
            # para cada valor del atributo seleccionamos el subconjunto Sv de S tal que la clase C ocurre en Sv 
            #se filtran los ejemplos para los que el atributo presenta el valor i y clase j
            Ocurrencias_clase_si_Valor=S.loc[ (S[A] == Valores_atributo.index[i]) & (S[A_clase] == Valores_clase.index[j])]
            
            C=Valores_clase.index[j]  
            # calculando Pi probabilidad de que aparesca el valor i y la clase j
            p=Ocurrencias_clase_si_Valor.count()/Valores_atributo[i]
            # Calculando Entropia para el Atributo cuando tiene valor i
            if float(p[0])<1.0 and float(p[0]>0.0):
                Entropia_A_con_Valor_i +=-(p[0]*np.log2(p[0]))
            else:Entropia_A_con_Valor_i=0
        # Calculando el la probabilidad de Valor
        P_valor_i=Valores_atributo[i]/S.count()
        # Se actualiza la ganancia de información
        Ganancia+=-P_valor_i[0]*Entropia_A_con_Valor_i
        Entropia_A_con_Valor_i=0
        
    return Ganancia
# Selecciona el atributo de con mayor ganacia de información
def Atributo_mejor_Ganancia(S, Atributo_clase, Atributos):
    Mejor_Atributo=""
    G=0
    for i in range(0,len(Atributos)):
        G_actual = Ganancia(S,Atributos[i],Atributo_clase)
        if G_actual>=G:
            Mejor_Atributo=Atributos[i]
            G=G_actual
    print("*****************Atributo que mejor clasifica es: "+Mejor_Atributo+" con ganancia de: "+str(G)+"**************************")
    return Mejor_Atributo
            
        
# Algoritmo ID3:
def ID3(S, Atributo_clase, Atributos,arbol,raiz,valor_raiz):
    
    Valores_clase=Hallar_Valores(Atributo_clase,S)
    #calculamos la probabilidad de clase en el conjunto S
    P_clases= Valores_clase/sum(Valores_clase)
    # Si todos los ejemplos son positivos devolver un nodo positivo
    # Si todos los ejemplos son negativos devolver un nodo negativo 
    for i in  range(0,P_clases.count()):
        if P_clases[i]==1:
            # se genera un identificador aleatorio para la hoja de clasificación
            ident=str(random.randint(0,15000000))
            arbol.node(ident,label=P_clases.index[i])
            arbol.edge(raiz,ident,label=valor_raiz)
            print("+++++++++++++++++Clasificó con clase "+P_clases.index[i]+"+++++++++++++++++++")
            return arbol
    # Si Atributos está vacío devolver un nodo con el voto mayoritario 
    # del valor del atributo objetivo en Ejemplos 
    if(len(Atributos)==0):
        P_clases.sort(axis=0, ascending=False, kind='quicksort', na_position='last', inplace=True)
        ident=str(random.randint(0,15000000))
        arbol.node(ident,label=P_clases.index[0])
        arbol.edge(raiz,ident,label=valor_raiz)
        print("+++++++++++++++++Clasificó con clase "+P_clases.index[0]+"+++++++++++++++++++")
        return arbol
    # Sea la Raíz del sub arbol el Aaributo que MEJOR clasifica a Ejemplos
    A=Atributo_mejor_Ganancia(S, Atributo_clase, Atributos)
   
    Valores_A=Hallar_Valores(A,S)
    arbol.node(A)
    #para valor de A adicionar a la raiz
    arbol.edge(raiz,A,valor_raiz)
    
    Atributos.remove(A)
    print("A eliminar"+A)
    #
    for i in range(0,Valores_A.count()):
        # S_en_A es el subconjunto de S (Ejemplos) donde el atributo A toma valor
        S_en_A= S.loc[S[A]== Valores_A.index[i]]
        # Caso base se logra clasificar por tanto es una hoja.
        if S_en_A.shape[0]==0:
            P_clases.sort(axis=0, ascending=False, kind='quicksort', na_position='last', inplace=True)
            str(ident=random.randint(0,15000000))
            arbol.node(ident,label=P_clases.index[0])
            arbol.edge(raiz,ident,label=valor_raiz)
            print("+++++++++++++++++Clasificó con clase "+P_clases.index[0]+"+++++++++++++++++++")
            return arbol
        else:
            S_en_A.drop(A, axis=1, inplace=True)
            print("Subconjunto para valor: "+Valores_A.index[i] +" del atributo "+A)
            print(S_en_A)
            # se realiza la llamada recursiva para S=S_en_A , raiz=A,valor_raiz=valor_i, atributos-A, 
            ID3(S_en_A, Atributo_clase, Atributos,arbol,A,Valores_A.index[i])
                
Atributos=list(emergencias.columns)
atributo_clase= Atributos.pop()
# Generando el arbol de desición
arbol = gv.Digraph(format='svg')

ID3(emergencias,atributo_clase,Atributos,arbol,'Árbol de Desición ID3: Por Roides Javier Cruz Lara', "")
arbol.render("arbol")
   
#Gain=Ganancia(emergencias,"Presion arterial","Administrar farmaco F")
#print(Gain)
#Atributos





/usr/local/lib/python3.5/dist-packages/ipykernel/__main__.py:3: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  app.launch_new_instance()
/usr/local/lib/python3.5/dist-packages/pandas/core/frame.py:3697: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


*****************Atributo que mejor clasifica es: Presion arterial con ganancia de: 0.2727875323887449**************************
A eliminarPresion arterial
Subconjunto para valor: Alta del atributo Presion arterial
   Azucar en sangre Indice de colesterol Alergia a antibioticos Otras alergia  \
0              Alto                 Alto                     No            No   
1              Alto                 Alto                     Si            No   
6              Bajo                 Alto                     Si            No   
7              Bajo                 Bajo                     No            Si   
8              Alto                 Bajo                     Si            Si   
11             Bajo                 Alto                     Si            Si   

   Administrar farmaco F  
0                     Si  
1                     Si  
6                     Si  
7                     Si  
8                     No  
11                    No  
*****************Atributo qu

*****************Atributo que mejor clasifica es: Alergia a antibioticos con ganancia de: 0.9182958340544896**************************
A eliminarAlergia a antibioticos
Subconjunto para valor: Si del atributo Alergia a antibioticos
   Azucar en sangre Indice de colesterol Administrar farmaco F
8              Alto                 Bajo                    No
11             Bajo                 Alto                    No
+++++++++++++++++Clasificó con clase No+++++++++++++++++++
Subconjunto para valor: No del atributo Alergia a antibioticos
  Azucar en sangre Indice de colesterol Administrar farmaco F
7             Bajo                 Bajo                    Si
+++++++++++++++++Clasificó con clase Si+++++++++++++++++++
Subconjunto para valor: Baja del atributo Presion arterial
   Azucar en sangre Indice de colesterol Alergia a antibioticos Otras alergia  \
2              Alto                 Bajo                     No            No   
5              Bajo                 Alto              

*****************Atributo que mejor clasifica es: Indice de colesterol con ganancia de: 0.9182958340544896**************************
A eliminarIndice de colesterol
Subconjunto para valor: Alto del atributo Indice de colesterol
  Azucar en sangre Alergia a antibioticos Otras alergia Administrar farmaco F
3             Alto                     No            Si                    No
4             Bajo                     Si            Si                    No
+++++++++++++++++Clasificó con clase No+++++++++++++++++++
Subconjunto para valor: Bajo del atributo Indice de colesterol
   Azucar en sangre Alergia a antibioticos Otras alergia Administrar farmaco F
10             Bajo                     Si            Si                    Si
+++++++++++++++++Clasificó con clase Si+++++++++++++++++++


'arbol.svg'

<img style="margin:0px auto;display:block" src="arbol.svg"/>

# Diferencias entre entre algoritmos J48 e ID3
No se puede aplicar ID3 a Zoo porque este solo funciona con Atributos objetivo binarios.

## ID3
<ol>
<li>Solo funciona con Atributos objetivo binarios y valores nominales</li>
<li>Recorre el espacio de búsqueda completo</li>
<li>Genera el árbol mediante un recorrido primero en profundidad</li>
<li>Un único árbol candidato en cada paso </li>
<li>Es un algoritmo muy rápido</li>
<li>Construye un árbol pequeño </li>
<li>Sin retroceso (peligro de óptimos locales), búsqueda en escalada </li>
<li>Decisiones tomadas a partir de conjuntos de ejemplos</li>
<li>Es fácil incurrir en un sobre entrenamiento o una sobre clasificación.</li>
<li>Sólo se comprueba un atributo en cada paso. </li>
</ol>

## J48
<ol>
<li>Ambos atributos discretos y continuos son manejados por este algoritmo (realiza dizcretización por umbrales)</li>
<li>Este algoritmo también maneja los valores faltantes en los datos de entrenamiento</li>
<li>Genera el árbol mediante un recorrido primero en profundidad</li>
<li>Realiza algoritmos backtracking mediante la tecnica de ramificación y poda:Después de que el árbol está completamente construido, este algoritmo regresa para eliminar las ramas que no ayudan a alcanzar los nodos de hoja.Optimiza la clasificación </li>
<li>Presenta mayor complejidad temporal y espacial</li>
<li>Al realizar discretización puede haber pérdida de información por lo que puede afectar la clasificación</li>

</ol>



# Comparación de eficacia en la clasificación de j48 con naive bayes
A continuación se muestran las capturas de pantallas resultantes de la aplicación de NB al dataset Zoo
<br>
<br>
<img style="margin:0px auto;display:block" src="Zoo-NB.png"/>
<br>
<br>
El NB acierta con 96% aproximandamente. Ahora observemos a J48
<br>
<br>
<img style="margin:0px auto;display:block" src="Zoo-J48.png"/>
<br>
<br>

En este caso podemos observar que NB clasifica con mayor certeza. No obstante J48 es un algoritmo mucho más sofisticado y puede ser aplicado a conjuntos de datos de diferente naturaleza de tipos a diferencia de NB que además pasa por alto las relaciones entre los atributos.

